In [31]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import seaborn as sns


##forcast solar energy generation
#put into optimiser -> battery schedule
#do exploratory analysis
#dic of vars(nas, pattern of na, odd values, errors, avg generation value[all generator])
#assume that audience has 1st year level -> doc through all the working process
#build model can use external factors to predict
#ex: temp forcast to foracst solar energy generation -> prototype
#goals: building models and evaluate all models that you've built(dif model can apply for dif vars->experiment and combine)
#keep track all models (how far predicted values from actual values)
#finalising report and presentation
#docs all info and keep clean github
#time series data
#use phase 1+2 as training(forecast)
#test data to test model
#tuning parameters
#multiverate, correlation

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
temp = pd.read_csv('/content/drive/MyDrive/Solar Farm Data/ERA5_Weather_Data_Monash.csv')#reading temp dataset

In [34]:
temp

,datetime (UTC),"coordinates (lat,lon)",model (name),model elevation (surface),utc_offset (hrs),temperature (degC),dewpoint_temperature (degC),wind_speed (m/s),mean_sea_level_pressure (Pa),relative_humidity ((0-1)),surface_solar_radiation (W/m^2),surface_thermal_radiation (W/m^2),total_cloud_cover (0-1)
0,2010-01-01 00:00:00,"(-37.91, 145.13)",era5,69.59,10.0,18.26,16.39,2.60,101046.38,0.89,287.01,408.35,1.00
1,2010-01-01 01:00:00,"(-37.91, 145.13)",era5,69.59,10.0,18.67,16.29,2.91,101037.96,0.86,360.79,411.02,1.00
2,2010-01-01 02:00:00,"(-37.91, 145.13)",era5,69.59,10.0,18.16,15.89,3.26,101017.26,0.87,291.54,410.67,1.00
3,2010-01-01 03:00:00,"(-37.91, 145.13)",era5,69.59,10.0,18.46,15.33,3.17,101022.56,0.82,357.11,410.95,1.00
4,2010-01-01 04:00:00,"(-37.91, 145.13)",era5,69.59,10.0,18.53,15.11,2.95,100940.03,0.80,459.91,410.00,0.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...
100052,2021-05-31 20:00:00,"(-37.91, 145.13)",era5,69.59,10.0,8.31,3.28,4.91,102033.73,0.71,0.00,278.11,0.93
100053,2021-05-31 21:00:00,"(-37.91, 145.13)",era5,69.59,10.0,8.15,3.16,4.63,102033.49,0.71,0.00,277.98,1.00
100054,2021-05-31 22:00:00,"(-37.91, 145.13)",era5,69.59,10.0,8.72,3.31,5.29,102069.84,0.69,8.99,281.65,1.00
100055,2021-05-31 23:00:00,"(-37.91, 145.13)",era5,69.59,10.0,9.83,3.88,5.60,102077.12,0.66,80.46,296.20,1.00


In [44]:
temp=temp.set_index(pd.DatetimeIndex(temp['datetime (UTC)'])).drop(temp.columns[0], axis=1)

In [46]:
temp=temp.dropna()#dropping na

In [45]:
temp.head()

,"coordinates (lat,lon)",model (name),model elevation (surface),utc_offset (hrs),temperature (degC),dewpoint_temperature (degC),wind_speed (m/s),mean_sea_level_pressure (Pa),relative_humidity ((0-1)),surface_solar_radiation (W/m^2),surface_thermal_radiation (W/m^2),total_cloud_cover (0-1)
datetime (UTC),,,,,,,,,,,,
2010-01-01 00:00:00,"(-37.91, 145.13)",era5,69.59,10.0,18.26,16.39,2.60,101046.38,0.89,287.01,408.35,1.0
2010-01-01 01:00:00,"(-37.91, 145.13)",era5,69.59,10.0,18.67,16.29,2.91,101037.96,0.86,360.79,411.02,1.0
2010-01-01 02:00:00,"(-37.91, 145.13)",era5,69.59,10.0,18.16,15.89,3.26,101017.26,0.87,291.54,410.67,1.0
2010-01-01 03:00:00,"(-37.91, 145.13)",era5,69.59,10.0,18.46,15.33,3.17,101022.56,0.82,357.11,410.95,1.0
2010-01-01 04:00:00,"(-37.91, 145.13)",era5,69.59,10.0,18.53,15.11,2.95,100940.03,0.80,459.91,410.00,0.9


In [47]:
temp.columns

Index(['coordinates (lat,lon)', 'model (name)', 'model elevation (surface)',
       'utc_offset (hrs)', 'temperature (degC)', 'dewpoint_temperature (degC)',
       'wind_speed (m/s)', 'mean_sea_level_pressure (Pa)',
       'relative_humidity ((0-1))', 'surface_solar_radiation (W/m^2)',
       'surface_thermal_radiation (W/m^2)', 'total_cloud_cover (0-1)'],
      dtype='object')

In [48]:
#temp = temp.set_index('datetime (UTC)').sort_index()
#temp_phase1 = temp[pd.Timestamp("2018-12-31 00:00:00"):pd.Timestamp("2020-09-30 00:00:00")]
StartDate ="2018-12-31 00:00:00"
EndDate = "2020-09-30 00:00:00"
temp_phase1 = temp[pd.Timestamp("2018-12-31 00:00:00"):pd.Timestamp("2020-09-30 00:00:00")]

In [49]:
temp_phase1

,"coordinates (lat,lon)",model (name),model elevation (surface),utc_offset (hrs),temperature (degC),dewpoint_temperature (degC),wind_speed (m/s),mean_sea_level_pressure (Pa),relative_humidity ((0-1)),surface_solar_radiation (W/m^2),surface_thermal_radiation (W/m^2),total_cloud_cover (0-1)
datetime (UTC),,,,,,,,,,,,
2018-12-31 00:00:00,"(-37.91, 145.13)",era5,69.59,10.0,21.03,13.77,2.03,101245.76,0.63,761.10,354.10,0.75
2018-12-31 01:00:00,"(-37.91, 145.13)",era5,69.59,10.0,21.61,13.79,2.19,101184.59,0.61,771.92,366.34,0.67
2018-12-31 02:00:00,"(-37.91, 145.13)",era5,69.59,10.0,22.56,14.25,2.42,101146.96,0.59,868.18,366.61,0.25
2018-12-31 03:00:00,"(-37.91, 145.13)",era5,69.59,10.0,23.61,14.56,2.90,101099.28,0.57,1017.15,369.25,0.24
2018-12-31 04:00:00,"(-37.91, 145.13)",era5,69.59,10.0,23.51,14.58,3.58,101004.24,0.57,964.11,367.96,0.19
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-29 20:00:00,"(-37.91, 145.13)",era5,69.59,10.0,12.93,8.68,4.63,101291.74,0.75,0.00,347.80,1.00
2020-09-29 21:00:00,"(-37.91, 145.13)",era5,69.59,10.0,12.50,9.19,4.63,101304.70,0.80,8.78,350.45,1.00
2020-09-29 22:00:00,"(-37.91, 145.13)",era5,69.59,10.0,12.81,10.26,5.10,101300.07,0.84,42.57,350.37,1.00
